In [ ]:
import sys
sys.path.append('src/')
%load_ext autoreload
%autoreload 2
sys.dont_write_bytecode = True
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import transformers
import torch
import transformers 
import torch
from util import unshard_deepspeed, generate_text
import yaml
from util import TokenizerTokens
from pathlib import Path
from model import GeneratorModel

In [3]:
from util import generate_prompt

texts = []

texts.append(
    generate_prompt(
        instruction="Think of a way to use the following material to build a shelter.",
        input="sticks and a piece of cloth",
    )
)

texts.append(
    generate_prompt(
        instruction="How can you stack the objects in a stable manner?",
        input="a book, 9 eggs, a nail, a plant, a shoe",
    )
)

texts.append(
    generate_prompt(
        instruction="Which is the third largest city in Sweden, list the first and second as well",
        input="",
    )
)

texts.append(
    generate_prompt(
        instruction="Write a poem describing my love for my wife",
        input="We meet 10 years ago and we moved in short after",
    )
)

texts.append(
    generate_prompt(
        instruction="Who is the prime minister in Sweden?",
        input="",
    )
)

texts.append(
    generate_prompt(
        instruction="Make up a game for two people that likes math and physics, make some examples as well",
        input="",
    )
)

texts.append(
    generate_prompt(
        instruction="Write a python program that solves the equation, it should take the two variables as float inputs and return the result",
        input="a + b * a",
    )
)

texts.append(
    generate_prompt(
        instruction="How old is my sister. Describe your reasoning",
        input="When I was six my sister was half my age, now I'm 60 years old",
    )
)

## My LORA trained LLAMA

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from util import get_ml_client, download_model
ml_client = get_ml_client()

download_model(
    ml_client=ml_client,
    name="alpacha_lora",
    destination="weights/alpacha_lora"
)

In [7]:
conf = yaml.safe_load(Path("config.yaml").read_text())
conf["model"]["load_in_8bit"] = True
conf["model"]["device_map"] = "auto"

model = GeneratorModel(**conf["model"], torch_dtype=torch.float16)

tokenizer = transformers.LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf"
)

tokenizer.add_special_tokens(
    {
        "eos_token": TokenizerTokens.DEFAULT_EOS_TOKEN,
        "bos_token": TokenizerTokens.DEFAULT_BOS_TOKEN,
        "unk_token": TokenizerTokens.DEFAULT_UNK_TOKEN,
        "pad_token": TokenizerTokens.DEFAULT_PAD_TOKEN,
    }
)

model.load_lora_pretrained("weights/alpacha_lora/adapter_model.bin")

for text in texts:

    response = generate_text(
        model=model,
        tokenizer=tokenizer,
        text=text,
        max_length=150,
        device="cuda",
    )

    print(response+ "\n\n" +  "-" * 40 +  "\n\n")

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Think of a way to use the following material to build a shelter.

### Input:
sticks and a piece of cloth

### Response:You can create a simple lean-to by using sticks as poles and wrapping them around a tree or other structure. The stick will hold up the roof while the cloth will provide protection from rain and wind.

----------------------------------------


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How can you stack the objects in a stable manner?

### Input:
a book, 9 eggs, a nail, a plant, a shoe

### Response:The egg should be placed on top of the book and then the shoes could go next to it. The plant would need to be placed at the bottom of the pile. Then the nails could be

In [ ]:
from util import generate_prompt
from model import generate_next_tokens

text = generate_prompt(
    instruction="Think of a way to use the following material to build a shelter.",
    input="sticks and a piece of cloth",
)

_ = generate_next_tokens(
    model=model,
    promt=text,
    tokenizer=tokenizer,
    top_p=0.95,
    temperature=0.9,
    max_length=200,
    device="cuda",
) 

## Baseline LLAMA

In [3]:
conf = yaml.safe_load(Path("config.yaml").read_text())

conf["model"]["load_in_8bit"] = True
conf["model"]["device_map"] = "auto"
conf["model"]["lora"] = False

model = GeneratorModel(**conf["model"], torch_dtype=torch.float16)

tokenizer = transformers.LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf"
)

tokenizer.pad_token = tokenizer.eos_token

for text in texts:

    response = generate_text(
        model=model,
        tokenizer=tokenizer,
        text=text,
        max_length=150,
        device="cuda",
    )

    print(response+ "\n\n" +  "-" * 40 +  "\n\n")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Think of a way to use the following material to build a shelter.

### Input:
sticks and a piece of cloth

### Response:
You can make a tent by using sticks as poles and tying them together at their ends. Then you can put your sleeping bag on top of this structure. You could also tie some branches or leaves onto it for extra support. If there are no trees around where you live, then you might have to go out into the woods in search of suitable materials.
\end{code}

----------------------------------------


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How can you stack the objects in a stable manner?

### Input:
a book, 9 eggs, a nail, a plant, a shoe

### Response:
You should place t

## Some Huggingface LORA trained LLAMA ...

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")

model = LlamaForCausalLM.from_pretrained(
    "chainyo/alpaca-lora-7b",
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

for text in texts:

    response = generate_text(
        model=model,
        tokenizer=tokenizer,
        text=text,
        max_length=150,
        device="cuda",
    )

    print(response+ "\n\n" +  "-" * 40 +  "\n\n")

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Think of a way to use the following material to build a shelter.

### Input:
sticks and a piece of cloth

### Response: 
The sticks can be used as poles for building a frame structure while the piece of cloth could be used as sailsailsailasailsailasailsailsalsalsssassssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss

----------------------------------------


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How can you stack the objects in a stable manner?

### Input:
a book, 9 eggs, a nail, a plant, a shoe

### Response